In [2]:
pip install tensorflow==2.9.1

Note: you may need to restart the kernel to use updated packages.


DEPRECATION: pyodbc 4.0.0-unsupported has a non-standard version number. pip 24.1 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of pyodbc or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063


In [37]:
import numpy as np
import pandas as pd


In [38]:
import tensorflow as tf

In [39]:
dataset = pd.read_csv("sc_data.csv")

In [40]:
dataset.head()

,Product type,SKU,Price,Availability,Number of products sold,Revenue generated,Customer demographics,Stock levels,Lead times,Order quantities,...,Location,Lead time,Production volumes,Manufacturing lead time,Manufacturing costs,Inspection results,Defect rates,Transportation modes,Routes,Costs
0,haircare,SKU0,69.808006,55,802,8661.996792,Non-binary,58,7,96,...,Mumbai,29,215,29,46.279879,Pending,0.226410,Road,Route B,187.752075
1,skincare,SKU1,14.843523,95,736,7460.900065,Female,53,30,37,...,Mumbai,23,517,30,33.616769,Pending,4.854068,Road,Route B,503.065579
2,haircare,SKU2,11.319683,34,8,9577.749626,Unknown,1,10,88,...,Mumbai,12,971,27,30.688019,Pending,4.580593,Air,Route C,141.920282
3,skincare,SKU3,61.163343,68,83,7766.836426,Non-binary,23,13,59,...,Kolkata,24,937,18,35.624741,Fail,4.746649,Rail,Route A,254.776159
4,skincare,SKU4,4.805496,26,871,2686.505152,Non-binary,5,3,56,...,Delhi,5,414,3,92.065161,Fail,3.145580,Air,Route A,923.440632


In [41]:
target_col = "Manufacturing costs"
feature_col = "Production volumes"

In [42]:
X = dataset[feature_col].values.reshape(-1,1)
y = dataset[target_col]

In [43]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
X_scaled = scaler.fit(X)

In [44]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(dataset[feature_col],dataset[target_col],test_size = 0.2, random_state = 42)

In [45]:
model = tf.keras.Sequential([
    tf.keras.layers.Dense(64, activation = 'relu', input_dim = 1),
    tf.keras.layers.Dense(32, activation = 'relu'),
    tf.keras.layers.Dense(1)
])

In [46]:
model.compile(optimizer = 'adam', loss = 'mean_squared_error')

In [47]:
min_vol = dataset[feature_col].min()
max_vol = 1000
step_size = 10

In [48]:
min_cost = float("inf")
best_vol = None

In [50]:
for prod_vol in range(min_vol,max_vol +1, step_size):
    norm_prod = scaler.transform(np.array([[prod_vol]]))
    pred_cost = model.predict(norm_prod)
    if prod_vol == best_vol:
        best_cost = pred_cost[0][0]
    if pred_cost[0][0] >= 0:
        min_cost = pred_cost[0][0]
        best_vol = prod_vol
        
print(best_vol)
print(min_cost)
    
    

1/1 [==============================] - 0s 14ms/step
994
0.11321139
